In [1]:
import gradio as gr
import joblib
import pandas as pd
from datetime import datetime
import numpy as np
import re

# --- Features list matches the trained Pune model from trial.ipynb ---
FEATURES = [
    'T2M', 'RH2M', 'WS2M', 'PRECTOTCORR', 'PS',
    'CLRSKY_SFC_SW_DWN',
    'ALLSKY_KT',
    'Month', 'DayOfYear', 'Weekday'
]

def load_pune_model():
    """Loads the specific joblib model file for Pune."""
    filename = "pune_xgb_model.pkl"
    try:
        model = joblib.load(filename)
        return model
    except FileNotFoundError:
        print("Error: pune_xgb_model.pkl not found. Please train and save the model first.")
        return None

def predict_solar_output(T2M, RH2M, WS2M, PRECTOTCORR, PS, CLRSKY_SFC_SW_DWN, ALLSKY_KT, date_str):
    """Predicts solar output using the feature set for the Pune model."""
    # 1. Input Type Validation
    try:
        inputs = [T2M, RH2M, WS2M, PRECTOTCORR, PS, CLRSKY_SFC_SW_DWN, ALLSKY_KT]
        T2M, RH2M, WS2M, PRECTOTCORR, PS, CLRSKY_SFC_SW_DWN, ALLSKY_KT = map(float, inputs)
    except (ValueError, TypeError):
        return "❌ Error: All weather inputs must be valid numbers."

    # 2. Boundary Case Validation for each parameter
    # --- MODIFIED LINE: Temperature range is now -40°C to 70°C ---
    if not (-40 <= T2M <= 70):
        return f"❌ Error: Temperature ({T2M}°C) is out of the valid range (-40°C to 70°C)."
    if not (0 <= RH2M <= 100):
        return f"❌ Error: Humidity ({RH2M}%) is out of the valid range (0% to 100%)."
    if not (0 <= WS2M <= 40):
        return f"❌ Error: Wind Speed ({WS2M} m/s) is out of the valid range (0 to 40 m/s)."
    if not (0 <= PRECTOTCORR <= 500):
        return f"❌ Error: Precipitation ({PRECTOTCORR} mm) is out of the valid range (0 to 500 mm)."
    if not (90 <= PS <= 110):
        return f"❌ Error: Surface Pressure ({PS} kPa) is out of the valid range (90 to 110 kPa)."
    if not (0 <= CLRSKY_SFC_SW_DWN <= 12):
        return f"❌ Error: Clear Sky Radiation ({CLRSKY_SFC_SW_DWN}) is out of the valid range (0 to 12)."
    if not (0 <= ALLSKY_KT <= 1):
        return f"❌ Error: Clearness Index ({ALLSKY_KT}) must be between 0.0 and 1.0."

    model = load_pune_model()
    if model is None:
        return "❌ Error: Model file for Pune not found. Please run your training script."

    try:
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    except ValueError:
        return "❌ Invalid date format. Please use YYYY-MM-DD."

    # 3. Feature Generation
    input_dict = {
        'T2M': T2M,
        'RH2M': RH2M,
        'WS2M': WS2M,
        'PRECTOTCORR': PRECTOTCORR,
        'PS': PS,
        'CLRSKY_SFC_SW_DWN': CLRSKY_SFC_SW_DWN,
        'ALLSKY_KT': ALLSKY_KT,
        'Month': date_obj.month,
        'DayOfYear': date_obj.timetuple().tm_yday,
        'Weekday': date_obj.weekday()
    }

    # 4. Create DataFrame and Predict
    X = pd.DataFrame([input_dict], columns=FEATURES)

    try:
        prediction = model.predict(X)[0]
    except Exception as e:
        return f"❌ Prediction error: {str(e)}"

    return f"🌞 Predicted Solar Radiation for Pune: {prediction:.2f} kWh/m²"


def build_pune_interface():
    """Builds the Gradio UI specifically for Pune."""
    with gr.Blocks() as demo:
        gr.Markdown("# 🔆 Solar Radiation Prediction for Pune")
        gr.Markdown("Predict daily solar energy based on weather conditions.")

        with gr.Row():
            gr.Markdown("### City: Pune")
            date_input = gr.Textbox(label="Date (YYYY-MM-DD)", value=datetime.today().strftime("%Y-%m-%d"))
        with gr.Row():
            T2M = gr.Number(label="Temperature (°C)", value=30.0)
            RH2M = gr.Number(label="Humidity (%)", value=50.0)
            WS2M = gr.Number(label="Wind Speed (m/s)", value=2.0)
        with gr.Row():
            PRECTOTCORR = gr.Number(label="Precipitation (mm/day)", value=0.0)
            PS = gr.Number(label="Surface Pressure (kPa)", value=101.3)
            CLRSKY_SFC_SW_DWN = gr.Number(label="Clear Sky Radiation (kWh/m²)", value=5.5)
            ALLSKY_KT = gr.Number(label="Clearness Index (0 to 1)", value=0.65)

        predict_btn = gr.Button("Predict Solar Radiation")
        output = gr.Textbox(label="Prediction Output", lines=2)

        predict_btn.click(
            fn=predict_solar_output,
            inputs=[T2M, RH2M, WS2M, PRECTOTCORR, PS, CLRSKY_SFC_SW_DWN, ALLSKY_KT, date_input],
            outputs=output
        )
    return demo

if __name__ == "__main__":
    app = build_pune_interface()
    app.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://1f0f38ad311589ab38.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
